In [3]:
import re 
import tiktoken

# 切分数据集

In [4]:
file = '/Users/chenzhiwei/Downloads/麻醉/麻醉学-第3版杨拔贤/2024_07_09_002a177993bd97d1d6d7g/2024_07_09_002a177993bd97d1d6d7g.tex'
with open(file, 'r') as f:
    tex_content = f.read()

def split_tex_by_section(tex_content):
    sections = re.split('\\\\section\*', tex_content)
    # sections = ['\section*' + sec for sec in sections if sec]
    return sections

In [18]:
sections = split_tex_by_section(tex_content)
print(len(sections))

677


In [19]:
sections = [section.replace('\n', ' ').replace('\\', ' ') for section in sections]

In [20]:
sections[2]

'{第 3 版} 主 审 曾因明  主 编 杨拔贤 李文志  副主编 刘 进 姚尚龙 郭曲练 邓小明  “十二五”普通高等教䇌本科国家级规划教材  卫生部“十二五”规划教材  全国高等医药教材建设研究会 “十二五”规划教材全国高等学校教材  供基础、临床、预防、口腔医学类专业用    includegraphics[max width= textwidth, center]{2024_07_09_002a177993bd97d1d6d7g-002}  '

In [ ]:
'{图书在版编目（CIP）数据}\n麻醉科住院医师手册 / 鞠辉, 冯艺主编. 一北京:北京大学医学出版社, 2017.8\n\nISBN 978-7-5659-1556-7\n\nI . (1)麻… II . (1)鞠… (2)冯 III . (1)麻醉学 - 手册\n\nIV . (1) R614-62\n\n中国版本图书馆 CIP 数据核字（2017）第 024842 号\n\n麻醉科住院医师手册\n\n主 编: 鞠 辉 冯 艺\n\n出版发行: 北京大学医学出版社\n\n地址: (100191) 北京市海淀区学院路 38 号 北京大学医学部院内\n\n电话: 发行部 010-82802230; 图书邮购 010-82802495\n\n网 址: http: \\href{//www.pumpress.com.cn}{//www.pumpress.com.cn}\n\nE-mail : \\href{mailto:booksale@bjmu.edu.cn}{booksale@bjmu.edu.cn}\n\n印刷: 北京佳信达欣艺术印刷有限公司\n\n经 销: 新华书店\n\n责任编辑: 王智敏 袁师军 责任校对：金軕文 责任印制：李啸\n\n开 本: $889 \\mathrm{~mm} \\times 1194 \\mathrm{~mm} \\quad 1 / 32$ 印张: 11.75 字数: 283 千字\n\n版 次: 2017 年 8 月第 1 版 2017 年 8 月第 1 次印刷\n\n书 号: ISBN 978-7-5659-1556-7\n\n定 价: 55.00 元\n\n版权所有, 违者必究\n\n(凡属质量问题请与本社发行部联系退换)\\\\\n有书由\n\n北京大学医学科学出版基金资助出版\n\n'

In [21]:
def num_tokens_from_messages(messages, model="gpt-4-0314"):
    """Return the number of tokens used by a list of messages."""
    try:
        encoding = tiktoken.encoding_for_model(model)
    except KeyError:
        print("Warning: model not found. Using cl100k_base encoding.")
        encoding = tiktoken.get_encoding("cl100k_base")
    if model in {
        "gpt-3.5-turbo-0613",
        "gpt-3.5-turbo-16k-0613",
        "gpt-4-0314",
        "gpt-4-32k-0314",
        "gpt-4-0613",
        "gpt-4-32k-0613",
        }:
        tokens_per_message = 3
        tokens_per_name = 1
    elif model == "gpt-3.5-turbo-0301":
        tokens_per_message = 4  # every message follows <|start|>{role/name}\n{content}<|end|>\n
        tokens_per_name = -1  # if there's a name, the role is omitted
    elif "gpt-3.5-turbo" in model:
        print("Warning: gpt-3.5-turbo may update over time. Returning num tokens assuming gpt-3.5-turbo-0613.")
        return num_tokens_from_messages(messages, model="gpt-3.5-turbo-0613")
    elif "gpt-4" in model:
        # print("Warning: gpt-4 may update over time. Returning num tokens assuming gpt-4-0613.")
        return num_tokens_from_messages(messages, model="gpt-4-0613")
    else:
        raise NotImplementedError(
            f"""num_tokens_from_messages() is not implemented for model {model}. See https://github.com/openai/openai-python/blob/main/chatml.md for information on how messages are converted to tokens."""
        )
    num_tokens = 0
    for message in messages:
        num_tokens += tokens_per_message
        for key, value in message.items():
            num_tokens += len(encoding.encode(value))
            if key == "name":
                num_tokens += tokens_per_name
    num_tokens += 3  # every reply is primed with <|start|>assistant<|message|>
    return num_tokens


In [22]:
with open("prompt/prompt1.md", 'r') as f:
    cn_prompt = f.read()
messages = [{
    "role": "system",
    "content": cn_prompt,
}]
num_tokens_from_messages(messages, "gpt-4")

592

In [42]:
def make_datasets(sections):
    datasets = []
    current_set = " "
    current_tokens = 0

    for section in sections:
        messages = [{
            "role": "system",
            "content": section,
        }]
        tokens = num_tokens_from_messages(messages, "gpt-4")
        if current_tokens + tokens <= 2000:
            current_set += str(section)
            current_tokens += tokens
        else:
            datasets.append(current_set)
            current_set = " " + section
            current_tokens = tokens

    if current_set:
        datasets.append(current_set)

    return datasets

datasets = make_datasets(sections)

In [43]:
import pickle

In [44]:
with open('datasets.pkl', 'wb') as f:
    pickle.dump(datasets, f)

In [45]:
print(datasets[13])

 {（一）麻醉科门诊} 随着医院管理工作的进步, 特别是保证质量、提高效率和减轻患者负担, 麻醉科门诊 (或麻醉前评估中心) 将日益成为医院门诊工作的重要组成部分。麻醉科门诊的主要工作内容如下:   begin{enumerate}    item 麻醉前检查、评估与准备 为缩短患者的住院周期 (床位周转率), 保证麻醉前充分准备, 凡拟接受择期手术的患者,在手术医师进行术前检查与准备的基础上, 人院前应由麻醉科医师在麻醉科门诊按要求作进一步的检查与准备。其优点是: (1)患者人院后即可安排手术, 甚至在当日即可安排手术, 可显著缩短住院日期, 提高床位周转率; (2)可避免因麻醉前检查不全面而延迟手术; (3)杜绝外科医师与麻醉科医师因对术前准备的意见不一致而发生矛盾; (4) 患者人院前麻醉科已能了解到病情及麻醉处理的难度, 便于恰当地安排麻醉工作。麻醉前检查、评估与准备工作目前均在病房进行, 随着医院现代化进程的加速, 有条件的医院应逐步将这一工作转移到门诊。     item 对麻醉并发症的随访和诊疗 麻醉后并发症由麻醉科医师亲自诊治是十分必要的。目前, 麻醉并发症的诊治并不是由麻醉科医师负责, 尤其是在患者出院后, 麻醉科医师无机会对这些患者进行诊疗, 疗效也不理想。随着麻醉科门诊的建立, 将改变这种状况, 对患者是有益的。     item 麻醉前会诊或咨询。     item 呼吸治疗、药物依赖戒断治疗 (“戒毒”)等。     item 疼痛诊疗可单独开设疼痛诊疗门诊或多学科疼痛诊疗中心,并可建立相应的病房。   end{enumerate}  {（二）临床麻醉} 临床麻醉的工作场所主要在手术室内, 目前已拓展到手术室外,其发展迅速,已成为临床麻醉的一个重要分支。手术室外麻醉广义是指病房手术室外的麻醉处理，包括门诊手术。狭义是门诊 (急诊) 及病房手术室外的麻醉、镇痛与镇静，包括介人治疗、内镜检查及各科无痛治疗等。在规模较大、条件较好的麻醉科,还应建立临床麻醉的分支学科 (或称为亚科),如心血管外科、胸外科、脑外科、产科和小儿外科麻醉等,以培养专门人才,提高专科麻醉的医疗质量。  


In [50]:
print(datasets[41])

 {（一）解剖} 支配下肢的神经来自于腰丛和骶丛神经。腰丛由腰,$_{1}  sim$ 腰 $_{4} left( mathrm{~L}_{1}  sim  mathrm{L}_{4} right)$ 前支构成, 常有胸 ${ }_{12}$ $ left( mathrm{T}_{12} right)$, 偶有腰 $ left( mathrm{L}_{5} right)$ 分支参与。由 $ mathrm{L}_{2}  sim  mathrm{L}_{4}$ 组成的腰丛成分主要支配大腿的前、内侧; $ mathrm{L}_{2}  sim  mathrm{L}_{4}$ 的前支组成闭孔神经, 后支组成股神经, 而 $ mathrm{L}_{2}$ 和 $ mathrm{L}_{3}$ 的后支又组成股外侧皮神经 (图 3-7)。腰丛神经位于腰大肌和腰方肌之间的腰大肌间隙内。   begin{center}  includegraphics[max width= textwidth]{2024_07_09_002a177993bd97d1d6d7g-055}  end{center}  图 3-7 腰神经丛解剖图  髊丛来源于骶 $ sim$ 骶 $ left( mathrm{S}_{1}  sim  mathrm{S}_{3} right)$ 骬神经和 $ mathrm{L}_{4}$ 和 $ mathrm{L}_{5}$ 前支的分支, 主要构成股后皮神经和坐骨神经, 一起经过坐骨大孔穿出骨盆, 支配下肢后面和足的运动和感觉。坐骨神经包含胫神经和腓总神经的主于 (由 $ mathrm{L}_{4} 、  mathrm{~L}_{5} 、  mathrm{~S}_{1}  sim  mathrm{S}_{3}$ 前支的腹侧支组成胫神经, 背侧支组成腓总神经), 二者在腘窝或腘窝上方从坐骨神经分出后, 胫神经走行在内侧而腓总神经绕到外侧下行。  {（二）腰神经丛阻滞(腰肌间祘阻滞) ( lumbar plexus block)}  begin{enumerate}    item 适应证 腰神经丛阻滞可同时阻滞股外侧皮神经、股神经和闭孔神经。因此,适用于膝部、大腿前部和䯘部手术; 加上坐骨神经阻滞可阻滞整个下肢; 置人导管可用于滕关节和骹关节的术后持续镇痛。     it

In [47]:
for i in range(len(datasets)):
    print(i, len(datasets[i]))

0 2984
1 1341
2 1342
3 1526
4 1357
5 1823
6 1294
7 759
8 2112
9 1150
10 932
11 953
12 1564
13 914
14 1720
15 1177
16 931
17 903
18 1400
19 2292
20 700
21 4673
22 1509
23 1586
24 1108
25 1253
26 415
27 1424
28 1884
29 1454
30 2056
31 1505
32 1493
33 1504
34 1209
35 1686
36 1061
37 1638
38 1693
39 1667
40 1433
41 1373
42 2047
43 1565
44 1520
45 1550
46 2046
47 1407
48 1594
49 1690
50 1562
51 1353
52 1286
53 1178
54 1152
55 1206
56 1653
57 1207
58 1769
59 574
60 3022
61 1613
62 1207
63 1156
64 2063
65 247
66 1800
67 785
68 1998
69 1828
70 1686
71 1578
72 1070
73 1543
74 2057
75 819
76 1628
77 1246
78 4413
79 1649
80 1088
81 799
82 2243
83 1838
84 1118
85 1544
86 1298
87 1868
88 1715
89 1723
90 1610
91 1070
92 1569
93 1548
94 1668
95 737
96 1613
97 1208
98 1521
99 1318
100 1586
101 1166
102 2426
103 1532
104 2090
105 1407
106 1839
107 1361
108 6268
109 2645
110 1440
111 3852
112 1007
113 899
114 1963
115 1262
116 933
117 1639
118 863
119 1985
120 886
121 2117
122 1557
123 537
124 4374
125 